# 맞춤형 State 작성

### 상태 (State)
- node 와 node 간에 정보를 전달할 때 State 객체데 담아 전달
- TypedDict : 파이썬 dict 에 타입 힌팅을 추가한 개념
- 모든 값을 다 채우지 않아도 된다.
- 새로운 node에서 값을 overwrite 방식으로 채운다.
- Reducer(add_messages 혹은 operator.add): 자동으로 list에 메시지를 추가해 주는 기능

In [1]:
from dotenv import load_dotenv

# API-KEY 읽어오기
load_dotenv()

True

In [2]:
from langchain.chat_models import init_chat_model

# 모델 초기화
llm = init_chat_model("openai:gpt-5-mini")
# llm = init_chat_model("google_genai:gemini-2.5-flash")

### 챗봇이 엔티티의 생일을 조사할 수 있도록 상태(state)에 name과 birthday 키를 추가합니다.

In [3]:
# LangGraph State 정의
from typing import Annotated               
from typing_extensions import TypedDict 
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]
    name: str
    birthday: str

### 도구 내부에서 State 업데이트
State 에 저장되기 전에 사람이 정보를 검토할 수 있도록 human_assistance 도구 안에서 State 키를 채워 넣습니다.  
이를 위해 도구 내부에서 Command를 사용해 상태 업데이트 명령을 발행합니다.

```
def human_assistance(
    name: str, birthday: str, tool_call_id: Annotated[str, InjectedToolCallId]) -> str: 
```
- tool_call_id: 도구 호출의 고유한 DI, 도구 호출과 그 결과를 추적하기 위해 LangChain이 자동 생성  
- 모델 프롬프트에 노출되지 않고, LangChain이 런타임에서 자동으로 넣어줍니다.  
- 덕분에 모델은 name, birthday만 보고 채워주면 되고, 내부 추적용 ID는 프레임워크가 관리합니다.

In [4]:
from langchain_core.messages import ToolMessage
from langchain_core.tools import InjectedToolCallId, tool
from langgraph.types import Command, interrupt

# human_assistance 도구 정의 (인간의 도움을 요청하는 도구)
# 상태 업데이트를 위한 ToolMessage를 생성하기 때문에 해당 도구 호출의 ID가 필요합니다.
# LangChain의 InjectedToolCallId를 사용하면 이 인자는 모델에게 노출되지 않습니다.
@tool
def human_assistance(
    name: str, birthday: str, tool_call_id: Annotated[str, InjectedToolCallId]) -> str: 
    
    """사람에게 도움을 요청합니다."""
    # 인간에게 질문과 현재 상태(name, birthday)를 전달 후 응답을 기다림
    human_response = interrupt(
        {
            "question": "이 내용이 맞나요?",  
            "name": name,
            "birthday": birthday,
        },
    )
    
    # 모델의 답변이 맞으면 (응답이 y로 시작하면) 현재 상태 유지
    if human_response.get("correct", "").lower().startswith("y"):
        verified_name = name
        verified_birthday = birthday
        response = "맞다고 응답"
    # 그렇지 않으면 인간 검토자가 제공한 수정된 name, birthday 사용
    else:
        verified_name = human_response.get("name", name)
        verified_birthday = human_response.get("birthday", birthday)
        response = f"수정됨 : {human_response}"

    # 도구 내부에서 ToolMessage를 사용해 상태(state)를 명시적으로 업데이트합니다.
    state_update = {
        "name": verified_name,
        "birthday": verified_birthday,
        "messages": [ToolMessage(response, tool_call_id=tool_call_id)],
    }
    # 도구 내부에서 Command 객체를 반환하여 State를 업데이트합니다
    return Command(update=state_update)

### Graph 생성

In [5]:
from langchain_tavily import TavilySearch
from langchain_core.tools import tool

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition

# 상태 기반 워크플로우 생성
graph_builder = StateGraph(State)
# 웹 검색을 도구
tavily = TavilySearch(max_results=5)
# 도구 리스트 (정보 추출 도구 추가)
tools = [tavily, human_assistance]

# LLM이 도구 호출 여부 판단
llm_with_tools = llm.bind_tools(tools)

# chatbot 노드 함수
def chatbot(state: State):
    message = llm_with_tools.invoke(state["messages"])
    # 병렬 도구 호출 비활성화: 인터럽트 후 툴 중복 호출 방지
    assert len(message.tool_calls) <= 1
    return {"messages": [message]}

# 워크플로우에 chatbot 노드 추가
graph_builder.add_node("chatbot", chatbot)

# tool 노드 워크플로우에 추가
tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

# 조건부 라우팅: tools로 이동하거나 END로 이동 (종료)
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

# tools 노드 실행 후 chatbot 노드로 다시 이동 (도구 결과 처리)
graph_builder.add_edge("tools", "chatbot")

# 워크플로우 시작점에서 chatbot 노드로 이동
graph_builder.add_edge(START, "chatbot")

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)
graph;

In [6]:
print(graph.get_graph().draw_ascii())

        +-----------+         
        | __start__ |         
        +-----------+         
               *              
               *              
               *              
          +---------+         
          | chatbot |         
          +---------+         
          .         .         
        ..           ..       
       .               .      
+---------+         +-------+ 
| __end__ |         | tools | 
+---------+         +-------+ 


### 챗봇에게 프롬프트를 보냅니다.
이순신 장군의 "생일(birthday)"을 조회하도록 챗봇에 지시하고, 필요한 정보를 얻으면 **human_assistance** 도구를 호출하도록 유도합니다.  

In [7]:
user_input = (
    "이순신 장군의 생일이 언제인가요?"
    "답을 얻으면 human_assistance 도구를 사용해 검토해 주세요."
)
config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config, 
    stream_mode="values"
)

for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

이순신 장군의 생일이 언제인가요?답을 얻으면 human_assistance 도구를 사용해 검토해 주세요.
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_RPTODv5x4CLhAPYglFHFDB0b)
 Call ID: call_RPTODv5x4CLhAPYglFHFDB0b
  Args:
    query: 이순신 생일 언제 태어난 날짜 1545 이순신 장군 생년월일
    search_depth: basic
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "이순신 생일 언제 태어난 날짜 1545 이순신 장군 생년월일", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://ko.wikipedia.org/wiki/%EC%9D%B4%EC%88%9C%EC%8B%A0", "title": "이순신 - 위키백과, 우리 모두의 백과사전", "content": "**이순신**(李舜臣, 1545년 4월 28일(윤음력 3월 8일)~1598년 12월 16일(음력 11월 19일)은 조선 중기 한국의 무신이다. 한산도대첩은 임진왜란의 3대 대첩 중 하나로, 1592년(선조 25) 8월 14일(음력 7월 8일) 한산도 앞바다에서 이순신 휘하의 조선 수군이 일본군을 크게 무찌른 해전이다. 이순신 함대는 1593년 3월 12일(음력 2월 10일) 웅천현 웅포로 진격하였다. 1. ↑ 정조실록 (1793년 7월

In [8]:
# 현재 스냅샷 조회
snapshot = graph.get_state(config)

# 필요한 키만 골라 보기 (예: name, birthday)
print("현재 상태:", {k: v for k, v in snapshot.values.items() if k in ("name", "birthday")})

현재 상태: {}


### 사람의 도움 추가
| 속성 이름              | 설명                                                               |
| ------------------ | ---------------------------------------------------------------- |
| `resume`           | 인터럽트나 외부 입력을 통해 **그래프 실행을 재개**할 때 사용하는 값                         |
| `update`           | 그래프의 상태(state)를 갱신할 데이터를 담는 필드                                   |


챗봇이 올바른 날짜를 식별하지 못했다고 가정하고, 필요한 정보를 제공해 주세요.

In [9]:
human_command = Command(
    resume={
        "name": "이순진 장군",
        "birthday": "Jan 17, 2000",
    }
)

# human_command = Command(
#     resume={
#         "correct": "yes",
#     },
# )


events = graph.stream(human_command, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  human_assistance (call_exdMKK7nHoJqOJnlEInFyXvN)
 Call ID: call_exdMKK7nHoJqOJnlEInFyXvN
  Args:
    name: 이순신
    birthday: 1545-04-28 (음력 3월 8일)
================================= Tool Message =================================
Name: human_assistance

수정됨 : {'name': '이순진 장군', 'birthday': 'Jan 17, 2000'}
================================== Ai Message ==================================

이순신 장군의 생일은
- 1545년 4월 28일 (음력 3월 8일)입니다.
(출처 예: 위키백과: https://ko.wikipedia.org/wiki/%EC%9D%B4%EC%88%9C%EC%8B%A0 , 한국민족문화대백과 등)

요청하신 대로 human_assistance 도구로 검토를 요청했는데, 도구의 응답이 잘못 수정된 결과를 반환했습니다:
- 제출한 값: 이름 = "이순신", 생일 = "1545-04-28 (음력 3월 8일)"
- human_assistance 결과(변경됨): {'name': '이순진 장군', 'birthday': 'Jan 17, 2000'}

의도한 검토가 제대로 이루어지지 않았습니다. 원하시면:
1) human_assistance에 동일한 검토 요청을 다시 보내 정확히 검사받도록 하겠습니다.  
2) 다른 공신력 있는 자료(국사편찬위원회, 한국학중앙연구원 등)를 추가로 찾아 근거를 더 제시하겠습니다.  
3) 이 상태로 마무리해 드리기를 원하시면 그대로 알려주세요.

어떤 방법으로 진행

In [10]:
# 실행 이후 상태 스냅샷 조회
snapshot = graph.get_state(config)

# 필요한 키만 골라 보기 (예: name, birthday)
print("현재 상태:", {k: v for k, v in snapshot.values.items() if k in ("name", "birthday")})

현재 상태: {'name': '이순진 장군', 'birthday': 'Jan 17, 2000'}


### 상태 수동 업데이트
LangGraph는 애플리케이션 상태를 매우 세밀하게 제어할 수 있는 기능을 제공합니다. 예를 들어, 실행 도중(인터럽트가 걸린 경우도 포함) 언제든지 graph.update_state를 사용하여 특정 키 값을 수동으로 덮어씌울 수 있습니다.

In [11]:
graph.update_state(config, {"name": "LangGraph (library)"})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0a24f1-bb7b-6b2e-8006-a9998ce56390'}}

In [12]:
# 실행 이후 상태 스냅샷 조회
snapshot = graph.get_state(config)

# 필요한 키만 골라 보기 (예: name, birthday)
print({k: v for k, v in snapshot.values.items() if k in ("name", "birthday")})

{'name': 'LangGraph (library)', 'birthday': 'Jan 17, 2000'}
